## Demand Forecasting

### Requirements


1. Plan an Approach - what steps to do you plan to follow and why? This can be high-level, but please include 1-2 paragraphs explaining your approach to a potential non-Data Scientist stakeholder.
2. Forecast Generation - produce a 28 day forecast (i.e. for each item_id predict demand for days d_1942 thru d_1969)
3. Forecast Evaluation - how will you evaluate your forecast peformance (how will this differ before/after going live)?
4. Prepare for Discussion - your submission will be the starting point for a discussion, be prepared to talk about your solution, assumptions/tradeoffs, expect questions on how this exercise is analogous to data problems faced at Shipbob.
5. Submit Response - See bottom section (either create git repo or email files


### PLAN


My typical approach would be:

1. Understand data and how to access it. The aim is to forecast demand based on historical data.
2. Exploration analysis: get an idea of data distribution, is there seasonalities? special days? some aspects that calls our attention?
3. Try some simple model to  get a baseline. Then increasinlgy add complexity to the model and iterate until metrics are good enough. Typically
   - xgboost, arima family, etc.

Prepare data for training forecasting. Idea is to training and compare multiple models. 

Preprocessing ideas: 

    - Prepare lag features.
    - Rolling features: e.g. moving average
    - Prepare target, predict next day? next week? average demand for the future 28 days?

In [3]:
# # Additional packages installed, added to requirements
# !pip install numpy=='1.26.4'
# !pip install jupyter_black
# !pip install category_encoders

In [4]:
%run ./commons.ipynb

In [5]:
from utils import (
    get_custom_calendar,
    get_input_data,
    prepare_datasets,
    plot_actual_vs_pred,
    compute_metrics,
    build_model_name,
)

import os
import joblib
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", None)
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

from process import *

# 2013-09-25, 2016-05-22
# NOTE: Last date is not included in time interval

TRAIN_START = 1
TRAIN_END = 1941
TRAIN_START_DATE = datetime.datetime.strptime("2011-01-29", "%Y-%m-%d")
TRAIN_END_DATE = datetime.datetime.strptime("2016-05-23", "%Y-%m-%d")


TEST_START = 1942
TEST_END = 1969
TEST_START_DATE = datetime.datetime.strptime("2016-05-23", "%Y-%m-%d")
TEST_END_DATE = datetime.datetime.strptime("2016-06-20", "%Y-%m-%d")


train_cols = [f"d_{i}" for i in range(1, 1942)]
test_cols = [f"d_{i}" for i in range(1942, 1970)]
print(len(train_cols), train_cols[:5], train_cols[-5:])
print(len(test_cols), test_cols[:5], test_cols[-5:])


print("Train Days:", (TRAIN_END_DATE - TRAIN_START_DATE).days, len(train_cols))
print("Test Days:", (TEST_END_DATE - TEST_START_DATE).days, len(test_cols))

1941 ['d_1', 'd_2', 'd_3', 'd_4', 'd_5'] ['d_1937', 'd_1938', 'd_1939', 'd_1940', 'd_1941']
28 ['d_1942', 'd_1943', 'd_1944', 'd_1945', 'd_1946'] ['d_1965', 'd_1966', 'd_1967', 'd_1968', 'd_1969']
Train Days: 1941 1941
Test Days: 28 28


In [6]:
calendar = get_custom_calendar()
print(calendar.shape)
calendar

2025-01-28 15:17:30.616 | DEBUG    | process:load_calendar:30 - Begin Loading Calendar Data...


(1969, 7)


,d,date,wm_yr_wk,year,wknu,event_name_1,event_type_1
0,d_1,2011-01-29,11101,11,1,nan,nan
1,d_2,2011-01-30,11101,11,1,nan,nan
2,d_3,2011-01-31,11101,11,1,nan,nan
3,d_4,2011-02-01,11101,11,1,nan,nan
4,d_5,2011-02-02,11101,11,1,nan,nan
...,...,...,...,...,...,...,...
1964,d_1965,2016-06-15,11620,16,20,nan,nan
1965,d_1966,2016-06-16,11620,16,20,nan,nan
1966,d_1967,2016-06-17,11620,16,20,nan,nan
1967,d_1968,2016-06-18,11621,16,21,nan,nan


In [7]:
# Parameters
train_state = "WI"
train_selected_item = "FOODS_3_823"

In [8]:
start_date = (TRAIN_END_DATE - datetime.timedelta(days=365 * 3)).strftime("%Y-%m-%d")


print(f"Start date used for training: {start_date}")
print(f"For item: {train_selected_item}")
print(f"For state: {train_state}")

Start date used for training: 2013-05-24
For item: FOODS_3_823
For state: WI


## 0. Prepare input data

In [9]:
# id_cols = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"]
prices = load_prices(PATH_INPUT)
data = load_sales(PATH_INPUT, prices)

2025-01-28 15:17:36.649 | DEBUG    | process:load_prices:45 - Begin Loading Price Data...
2025-01-28 15:17:38.135 | DEBUG    | process:load_sales:62 - Begin Loading Sales Data...


In [ ]:
# NOTES: custom functions ensures data is sorted
data = get_input_data(
    data=data,
    prices=prices,
    calendar=calendar,
    # start_date=start_date,
    state_id=train_state,
    item_id=train_selected_item,
    drop_columns=[],
)
print(data.shape)
data.dtypes
# CA, (7764, 13)

In [ ]:
data.head()

## 1. Feature Engineering

In [ ]:



def classify_columns(df):
    ds = df.dtypes.reset_index().assign(Dtype=lambda x: x[0].astype(str))
    features_types = ds.groupby("Dtype")["index"].apply(list).to_dict()
    features_types["num"] = []
    for k, v in features_types.items():
        if "float" in k or "int" in k:
            features_types["num"] += v
    return features_types

In [ ]:
print("Original data shape:", data.shape)

df = prepare_datasets(data)
df = df.dropna()

print("Clean data shape:", df.shape)
df.iloc[:10]

In [ ]:
idcols = ["id", "item_id", "dept_id", "cat_id", "date", "d", "wm_yr_wk"]
target = ["sales"]
features = [c for c in df.columns if c not in idcols + target]
features_types = classify_columns(df[features])

categorical_cols = features_types["category"]
numerical_cols = features_types["num"]

print(f"Categorical cols: {categorical_cols}")
print(f"Numerical cols: {numerical_cols}")

In [ ]:
def custom_train_test_split(X, y, test_size):
    """
    Perform train-test split keeping time series sorted
    """
    X_train = X.iloc[:-test_size]
    y_train = y.iloc[:-test_size]
    X_test = X.iloc[-test_size:]
    y_test = y.iloc[-test_size:]

    return X_train, X_test, y_train, y_test


X_train, X_test, y_train, y_test = custom_train_test_split(
    X=df.dropna().drop(columns=["sales"]), y=df.dropna()["sales"], test_size=int(0.3 * len(df))
)
print(X_train.shape, X_test.shape)

In [ ]:
X_test.head()

## 2. Model Training


In [ ]:
from typing import List
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from category_encoders import CatBoostEncoder
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor


def create_pipeline(numerical_columns: List[str], categorical_columns: List[str]):
    # Preprocessing:
    # Handle missing categorical values
    # Use CatBoostEncoder
    categorical_preprocessor = Pipeline(
        steps=[
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("encoder", CatBoostEncoder()),
        ]
    )

    numerical_preprocessor = Pipeline(
        steps=[
            ("imputer", SimpleImputer(strategy="mean")),
        ]
    )

    # Combine preprocessors in a ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numerical_preprocessor, numerical_columns),
            ("cat", categorical_preprocessor, categorical_columns),
        ]
    )

    rf_reg_params = {
        "bootstrap": True,
        "ccp_alpha": 0.0,
        "criterion": "squared_error",
        "max_depth": None,
        "max_features": 1.0,
        "max_leaf_nodes": None,
        "max_samples": None,
        "min_impurity_decrease": 0.0,
        "min_samples_leaf": 1,
        "min_samples_split": 2,
        "min_weight_fraction_leaf": 0.0,
        "monotonic_cst": None,
        "n_estimators": 150,
        "n_jobs": None,
        "oob_score": False,
        "random_state": 42,
        "verbose": 0,
        "warm_start": False,
    }

    pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor),
            # ("to_numpy", FunctionTransformer(lambda x: x.values)),  # Explicitly convert to NumPy array
            ("regressor", RandomForestRegressor(**rf_reg_params)),
        ]
    )

    return pipeline

In [ ]:
pipe = create_pipeline(
    categorical_columns=categorical_cols,
    numerical_columns=numerical_cols,
)
pipe

In [ ]:
model = pipe.fit(X_train, y_train)

### 2.1 Feature Importances

In [ ]:
fimp = pd.DataFrame(
    dict(
        features=[c for c in X_train.columns if c not in id_cols + ["wm_yr_wk"]],
        importance=model.steps[1][1].feature_importances_,
    )
).sort_values("importance", ascending=False)

fig, ax = plt.subplots(figsize=(10, 8))
_ = sns.barplot(fimp, y="features", x="importance", ax=ax, orient="h")

## 3. Model Assesment

In [ ]:
x_transform = model.steps[0][1].transform(X_train)
# just checking
# todo: we have to drop all those columns with std=0
pd.DataFrame(x_transform).describe().round(2)

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(12, 5))

# Train scores
y_pred = model.predict(X_train)
ax = plot_actual_vs_pred(y_train, y_pred, ax=axs[0])
ax.set_title("Train")
metrics_train = compute_metrics(y_train, y_pred)

# Test Metrics
y_test_pred = model.predict(X_test)
ax = plot_actual_vs_pred(y_test, y_test_pred, ax=axs[1])
ax.set_title("Test")
metrics_test = compute_metrics(y_test, y_test_pred)

**NOTES**

- Predictions are always below the max value of actual values saw durint training. This is expected due to characteristics of xgboost (tree based model).
- Predictions are always below the actual values meaning this model is understimated demand.
- Some improvements can be obtained if we optimized the metaparameters of the xgboost.

In [ ]:
pd.DataFrame({"train": metrics_train, "test": metrics_test}).T.round(3)

**NOTES**
- Clearly the model is overfitted. Results are much worst in the test scenario.

- First approach: Want to predict for 1 item in 1 store 

## 4. Save model

In [ ]:
train_state, train_selected_item

In [ ]:
model_name = build_model_name(state_id=train_state, item_tag=train_selected_item.split("_")[0])
print(model_name)
joblib.dump(model, model_name)


## 5. How To Continue... 

There are still many points to explore and improve:


1. Optimization of Hyperparameters: Implement GridSearch to tune the metaparameters of the Random Forest model.
2. Extend to Weekly/Monthly Demand Prediction: Adapt the framework to predict expected demand on a weekly or monthly basis.
3. Predict Demand for Multiple Items: Explore alternatives for predicting sales across all items, either by parallelizing the existing framework or retraining the model with all items for a given store or state.
5. Alternative Models: Explore the use of ARIMA for time series forecasting, as well as conduct analysis of seasonality and trends.
